In [7]:
library(lmtest)    # for tests after linear regression
library(sandwich)  # to make heteroskedastic variance matrix
library("multiwayvcov")  # to make variance matrix with clustering
library(stargazer)
#library('fastDummies')
library(IRdisplay)
library(car)

In [8]:
data_prep_hcq<- function(data){
    # Assuming 'data' is a valid data frame object
    
    
    X <- data #subset(data, state != "DISTRICT OF COLUMBIA" )# & (data$state != "TEXAS") &  (data$state != "FLORIDA") )
    #states_to_keep <- c('FLORIDA', 'TEXAS', 'CALIFORNIA', 'GEORGIA', 'NORTH CAROLINA', 'TENNESSEE', 'MISSOURI', 'ARIZONA', 'ILLINOIS', 'VIRGINIA', 'INDIANA', 'OHIO', 'SOUTH CAROLINA', 'PENNSYLVANIA', 'NEW YORK', 'ALABAMA', 'WASHINGTON', 'MICHIGAN', 'OKLAHOMA', 'NEW JERSEY', 'COLORADO', 'KANSAS', 'UTAH', 'MISSISSIPPI')
    # Subset the dataset based on the "state" variable
    #X <- X[X$state %in% states_to_keep, ]

    X$week_f = as.factor(X$year*100+X$week)

    #X$trump_binary_2016 =X$trump_share_2016
    
    X$trump_share = X$trump_share_2016
    X$trump_binary = X$trump_binary_2016
        
     X$trump_SP_prev <- X$trump_share * X$SP_prev
    X$trump_UMP_prev <- X$trump_share * X$UMP_prev
    X$trump_LMP_prev <- X$trump_share * X$LMP_prev
    X$trump_SP_rvsd <- X$trump_share * X$SP_rvsd
    X$trump_UMP_rvsd <- X$trump_share * X$UMP_rvsd
    X$trump_UMP_rvsd <- X$trump_share * X$UMP_rvsd
    X$trump_WP_prev <- X$trump_share * X$WP_prev

    

    return (X)

    }

In [23]:





regression_custom_event <- function(metric='first', metric_type ='per_capita') {
    models=c()
    se=c()
    
    drug='ivermectin'
    
    
    
    
    base_ =c( 'per_eff_sz', "week_f",'trump_share')
    base__ = c('cases')
    
    
    


    custom_summary <- function(x) {
  min_value <- min(x)
  max_value <- max(x)
  mean_value <- mean(x)
  median_value <- median(x)
  std_dev <- sd(x)
  
  summary_data <- data.frame(
    Minimum = min_value,
    Maximum = max_value,
    Mean = mean_value,
    Median = median_value,
    Standard_Deviation = std_dev
  )
  
  return(summary_data)
  }
    
  #summary_data <- custom_summary(X$per_eff_sz)

# Print the summary
#print(summary_data)  
    flush.console()
    
    
    
    
    for (duration in c('immediate','all','after')){
        
        file <- sprintf("../Processed Data/regression/IVM_Regression_Dataset_per_untransformed_%s_False_gpi.csv", metric)
        data <- read.csv(file = file)
        X = data_prep_hcq(data)
                if (metric_type == 'per_capita') {
                    X$per_eff_sz = ( (  ((X$att)/X$population )))*100000# (X$att/X$att_sd)#  # X$att_efctsz/X$att_efctsz_sd/X$populationX$att_sd

                    X$cases<- (X$cases/X$population)*100000
                    #X$cases_sq = X$cases*X$trump_share_2016
                }
    
   
    if ( (duration=='all') ){
         X= subset(X, ((year==2021) & (week>2) & (week<25) ) )
        }
   
    if ( (duration=='after') ){
         X= subset(X, ((year==2021) & (week>25) ) )
        }
    
    if ( (duration=='immediate') ){
         X= subset(X, ((year==2020) & (week>50) )|((year==2021) & (week<2) ) )
        }    
        
        base = c(base_,base__)
        
        X4=X
        #print("CHecking the data")
        #print(head(X4))
        print(unique(X4$week_f))
        #print(base)
        #flush.console()   
        
        
        
        Weight <- 1 / ((X4$att_sd*X4$att_sd*100000*100000)/(X4$population*X4$population))
        
        X5= X4[, (colnames(X4) %in% c(base))]

             
        model4 <- lm(per_eff_sz ~ ., data = X5)
        #cov1 = vcovCL(model4, cluster = X4$trump_binary_2016)  
        cov1         <- vcovHC(model4, type="HC1" ) #vcovCL(model4,  cluster = ~state, type = "HC1", sandwich = TRUE, fix = TRUE) #vcovHC(model1, type="HC1")## #vcovPL(glm.fit_0, cluster = ~ state+time, order.by = ~ time)#vcovCL(glm.fit_0, , cluster = ~time, type = "HC3", sandwich = TRUE, fix = TRUE) #vcovHC(glm.fit_0, type = "HC3")
        robust_se    <- sqrt(diag(cov1))
        se[[length(se)+1]]  = robust_se
        models[[length(models)+1]]   = model4  
        
  
   
        
        }
    print(base)
    flush.console()
    base = base[base != 'week_f']
   

    vars = c("Constant", base)#,policies, interactions)
    display_html(stargazer( models, 
                           star.char = c("*", "**", "***"),
              star.cutoffs = c(0.05, 0.01, 0.001),
                           intercept.bottom = FALSE, 
                           align=TRUE, 
                           notes = c( "Week and State Fixed Effects"), 
                           notes.append = T,
                           font.size ='small',
              title = paste(drug, metric,sep=''), 
                           keep = vars,  
                           type="html", 
                           #order = c(6,2,4,8,10,3,5,7,9,10,11),
                           se=se,
                           dep.var.labels = paste("Indexed Incremental", metric, metric_type, sep=' '), 
                           #covariate.labels = labels,
                           out=paste( "model_",drug,metric, ".doc", sep='_')))
    flush.console()
    
    

}

In [24]:
for (metric in c('claim')){
    #
    regression_custom_event(metric=metric,metric_type='per_capita')
    #regression_custom_hcq(metric=metric,metric_type='effect_size')
}


[1] 202051 202052 202101
92 Levels: 202013 202014 202015 202016 202017 202018 202019 202020 ... 202152
 [1] 202103 202104 202105 202106 202107 202108 202109 202110 202111 202112
[11] 202113 202114 202115 202116 202117 202118 202119 202120 202121 202122
[21] 202123 202124
92 Levels: 202013 202014 202015 202016 202017 202018 202019 202020 ... 202152
 [1] 202126 202127 202128 202129 202130 202131 202132 202133 202134 202135
[11] 202136 202137 202138 202139 202140 202141 202142 202143 202144 202145
[21] 202146 202147 202148 202149 202150 202151 202152
92 Levels: 202013 202014 202015 202016 202017 202018 202019 202020 ... 202152
[1] "per_eff_sz"  "week_f"      "trump_share" "cases"      


ivermectinclaim Dependent variable: Indexed Incremental claim per (1) (2) (3) Constant -1.242 0.342 1.432 (0.766) (0.611) (0.755) cases 0.007 *** 0.008 *** 0.014 *** (0.002) (0.002) (0.002) trump_share 0.742 0.098 2.895 *** (0.429) (0.130) (0.446) Observations 150 1,100 1,350 R 2 0.126 0.110 0.289 Adjusted R 2 0.102 0.091 0.274 Residual Std. Error 3.442 (df = 145) 2.657 (df = 1076) 8.920 (df = 1321) F Statistic 5.235 *** (df = 4; 145) 5.772 *** (df = 23; 1076) 19.217 *** (df = 28; 1321) Note: * p<0.05; ** p<0.01; *** p<0.001 Week and State Fixed Effects


<table style="text-align:center"><caption><strong>ivermectinclaim</strong></caption>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="3"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="3" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="3">Indexed Incremental claim per</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td></tr>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Constant</td><td>-1.242</td><td>0.342</td><td>1.432</td></tr>
<tr><td style="text-align:left"></td><td>(0.766)</td><td>(0.611)</td><td>(0.755)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">cases</td><td>0.007<sup>***</sup></td><td>0.008<sup>***</sup></td><td>0.014<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(0.00

In [11]:





regression_custom_event <- function(metric='first', metric_type ='per_capita') {
    models=c()
    se=c()
    
    drug='ivermectin'
    
    
    
    
    base_ =c( 'per_eff_sz', "week_f",'trump_share')
    base__ = c('cases')
    
    
    


    custom_summary <- function(x) {
  min_value <- min(x)
  max_value <- max(x)
  mean_value <- mean(x)
  median_value <- median(x)
  std_dev <- sd(x)
  
  summary_data <- data.frame(
    Minimum = min_value,
    Maximum = max_value,
    Mean = mean_value,
    Median = median_value,
    Standard_Deviation = std_dev
  )
  
  return(summary_data)
  }
    
  #summary_data <- custom_summary(X$per_eff_sz)

# Print the summary
#print(summary_data)  
    flush.console()
    
    
    
    
    for (duration in c('immediate','all','after')){
        
        file <- sprintf("../Processed Data/regression/IVM_Regression_Dataset_per_untransformed_%s_False_gpi.csv", metric)
        data <- read.csv(file = file)
        X = data_prep_hcq(data)
                if (metric_type == 'per_capita') {
                    X$per_eff_sz = ( (  ((X$att)/X$population )))*100000# (X$att/X$att_sd)#  # X$att_efctsz/X$att_efctsz_sd/X$populationX$att_sd

                    X$cases<- (X$cases/X$population)*100000
                    #X$cases_sq = X$cases*X$trump_share_2016
                }
    
   
    if ( (duration=='all') ){
         X= subset(X, ((year==2021) & (week>2) & (week<25) ) )
        }
   
    if ( (duration=='after') ){
         X= subset(X, ((year==2021) & (week>25) ) )
        }
    
    if ( (duration=='immediate') ){
         X= subset(X, ((year==2020) & (week>50) )|((year==2021) & (week<2) ) )
        }    
        
        base = c(base_,base__)
        
        X4=X
        #print("CHecking the data")
        #print(head(X4))
        print(unique(X4$week_f))
        #print(base)
        #flush.console()   
        
        
        
        Weight <- 1 / ((X4$att_sd*X4$att_sd*100000*100000)/(X4$population*X4$population))
        
        X5= X4[, (colnames(X4) %in% c(base))]

             
        model4 <- lm(per_eff_sz ~ ., data = X5)
        #cov1 = vcovCL(model4, cluster = X4$trump_binary_2016)  
        cov1         <- vcovHC(model4, type="HC1" ) #vcovCL(model4,  cluster = ~state, type = "HC1", sandwich = TRUE, fix = TRUE) #vcovHC(model1, type="HC1")## #vcovPL(glm.fit_0, cluster = ~ state+time, order.by = ~ time)#vcovCL(glm.fit_0, , cluster = ~time, type = "HC3", sandwich = TRUE, fix = TRUE) #vcovHC(glm.fit_0, type = "HC3")
        robust_se    <- sqrt(diag(cov1))
        se[[length(se)+1]]  = robust_se
        models[[length(models)+1]]   = model4  
        
  
   
        
        }
    print(base)
    flush.console()
    base = base[base != 'week_f']
   

    vars = c("Constant", base)#,policies, interactions)
    display_html(stargazer( models, 
                           star.char = c("*", "**", "***"),
              star.cutoffs = c(0.05, 0.01, 0.001),
                           intercept.bottom = FALSE, 
                           align=TRUE, 
                           notes = c( "Week and State Fixed Effects"), 
                           notes.append = T,
                           font.size ='small',
              title = paste(drug, metric,sep=''), 
                           keep = vars,  
                           type="html", 
                           #order = c(6,2,4,8,10,3,5,7,9,10,11),
                           se=se,
                           dep.var.labels = paste("Indexed Incremental", metric, metric_type, sep=' '), 
                           #covariate.labels = labels,
                           out=paste( "model_",drug,metric, ".doc", sep='_')))
    flush.console()
    
    

}

In [12]:
for (metric in c('claim')){
    #
    regression_custom_event(metric=metric,metric_type='per_capita')
    #regression_custom_hcq(metric=metric,metric_type='effect_size')
}


[1] 202051 202052 202101
92 Levels: 202013 202014 202015 202016 202017 202018 202019 202020 ... 202152
 [1] 202103 202104 202105 202106 202107 202108 202109 202110 202111 202112
[11] 202113 202114 202115 202116 202117 202118 202119 202120 202121 202122
[21] 202123 202124
92 Levels: 202013 202014 202015 202016 202017 202018 202019 202020 ... 202152
 [1] 202126 202127 202128 202129 202130 202131 202132 202133 202134 202135
[11] 202136 202137 202138 202139 202140 202141 202142 202143 202144 202145
[21] 202146 202147 202148 202149 202150 202151 202152
92 Levels: 202013 202014 202015 202016 202017 202018 202019 202020 ... 202152
[1] "per_eff_sz"  "week_f"      "trump_share" "cases"      


ivermectinclaim Dependent variable: Indexed Incremental claim per (1) (2) (3) Constant -1.242 0.342 1.432 (0.766) (0.611) (0.755) cases 0.007 *** 0.008 *** 0.014 *** (0.002) (0.002) (0.002) trump_share 0.742 0.098 2.895 *** (0.429) (0.130) (0.446) Observations 150 1,100 1,350 R 2 0.126 0.110 0.289 Adjusted R 2 0.102 0.091 0.274 Residual Std. Error 3.442 (df = 145) 2.657 (df = 1076) 8.920 (df = 1321) F Statistic 5.235 *** (df = 4; 145) 5.772 *** (df = 23; 1076) 19.217 *** (df = 28; 1321) Note: * p<0.05; ** p<0.01; *** p<0.001 Week and State Fixed Effects


<table style="text-align:center"><caption><strong>ivermectinclaim</strong></caption>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="3"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="3" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="3">Indexed Incremental claim per</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td></tr>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Constant</td><td>-1.242</td><td>0.342</td><td>1.432</td></tr>
<tr><td style="text-align:left"></td><td>(0.766)</td><td>(0.611)</td><td>(0.755)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">cases</td><td>0.007<sup>***</sup></td><td>0.008<sup>***</sup></td><td>0.014<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(0.00